### Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import os
import pathlib
from utils import *
import numpy as np
import cv2
from sklearn.utils import shuffle #shuffling the data improves the model

In [2]:
X, y = read_images()
X = crop_images(X)
X = resize_and_rescale(X)
X_train, y_train, X_test, y_test = split_data(X, y)
X_train, y_train = augment_data(X_train,y_train)

X_train = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_train])
X_test = np.array([[[[x,x,x]  for x in x2] for x2 in x1] for x1 in X_test])

X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

2022-04-21 11:12:21.731049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-21 11:12:22.657317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/miglou/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-21 11:12:22.657339: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-04-21 11:12:22.694459: I tensorflow/core/platform/cpu_feature_guard.cc:151] 

In [3]:
vgg16 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet")

vgg16.trainable = False

inputs = tf.keras.Input(shape=(224, 224,3))
x = tf.keras.applications.vgg16.preprocess_input(
    inputs, data_format=None)

x = vgg16(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
x = tf.keras.layers.Dense(4096, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 512)               0     

In [4]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10
13/13 [==============================] - 52s 4s/step - loss: 11.6053 - accuracy: 0.5965 - val_loss: 0.7865 - val_accuracy: 0.7451
Epoch 2/10
13/13 [==============================] - 48s 4s/step - loss: 0.4959 - accuracy: 0.7649 - val_loss: 0.4694 - val_accuracy: 0.7647
Epoch 3/10
13/13 [==============================] - 52s 4s/step - loss: 0.3438 - accuracy: 0.8515 - val_loss: 0.4409 - val_accuracy: 0.8039
Epoch 4/10
13/13 [==============================] - 59s 5s/step - loss: 0.2643 - accuracy: 0.9059 - val_loss: 0.3890 - val_accuracy: 0.8431
Epoch 5/10
13/13 [==============================] - 59s 5s/step - loss: 0.1909 - accuracy: 0.9233 - val_loss: 0.4191 - val_accuracy: 0.9412
Epoch 6/10
13/13 [==============================] - 59s 5s/step - loss: 0.1779 - accuracy: 0.9233 - val_loss: 0.3979 - val_accuracy: 0.9020
Epoch 7/10
13/13 [==============================] - 60s 5s/step - loss: 0.1664 - accuracy: 0.9381 - val_loss: 0.4782 - val_accuracy: 0.9020
Epoch 8/10
13/13 [=

In [5]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 7s 2s/step - loss: 0.4457 - accuracy: 0.8627


[0.4457293748855591, 0.8627451062202454]